# End to End example to manage lifecycle of ML models deployed on the edge using SageMaker Edge Manager + GreenGrass v2

**SageMaker Studio Kernel**: Data Science

## Contents 

* Use Case
* Workflow
* Setup
* Building and Deploying the ML Model
* Deploy Wind Turbine application to EC2 with Greengrass V2
* Cleanup


## Use Case

The challenge we're trying to address here is to detect anomalies in the components of a Wind Turbine. Each wind turbine has many sensors that reads data like:
 - Internal & external temperature
 - Wind speed
 - Rotor speed
 - Air pressure
 - Voltage (or current) in the generator
 - Vibration in the GearBox (using an IMU -> Accelerometer + Gyroscope)

So, depending on the types of the anomalies we want to detect, we need to select one or more features and then prepare a dataset that 'explains' the anomalies. We are interested in three types of anomalies:
 - Rotor speed (when the rotor is not in an expected speed)
 - Produced voltage (when the generator is not producing the expected voltage)
 - Gearbox vibration (when the vibration of the gearbox is far from the expected)
 
All these three anomalies (or violations) depend on many variables while the turbine is working. Thus, in order to address that, let's use a ML model called [Autoencoder](https://en.wikipedia.org/wiki/Autoencoder), with correlated features. This model is unsupervised. It learns the latent representation of the dataset and tries to predict (regression) the same tensor given as input. The strategy then is to use a dataset collected from a normal turbine (without anomalies). The model will then learn **'what is a normal turbine'**. When the sensors readings of a malfunctioning turbine is used as input, the model will not be able to rebuild the input, predicting something with a high error and detected as an anomaly.


## Workflow

In this example, you will create a robust end-to-end solution that manages the lifecycle of ML models deployed to a wind turbine fleet to detect the anomalies in the operation using SageMaker Edge Manager.

 - Prepare a ML model
     - download a pre-trained model;
     - compile the ML model with SageMaker Neo for Linux x86_64;
     - create a deployment package using SageMaker Edge Manager;
     - download/unpack the deployment package;
 - Download/unpack a package with the IoT certificates, required by the agent; 
 - Download/unpack **SageMaker Edge Agent** for Linux x86_64;
 - Generate the protobuf/grpc stubs (.py scripts) - with these files we will send requests via unix:// sockets to the agent; 
 - Using some helper functions, we're going to interact with the agent and do some tests.

The following diagram shows the resources, required to run this experiment and understand how the agent works and how to interact with it.  
![Pipeline](../imgs/EdgeManagerWorkshop_GGv2.png)

## Setup

We need an AWS account role with SageMaker access. This role is used to give SageMaker access to S3, launch an EC2 instance and create components and deployments in Greengrass.

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import botocore
import json
import time

role = get_execution_role()
sess = sagemaker.Session()
region = boto3.Session().region_name

# S3 bucket and folders for saving model artifacts.
# Feel free to specify different bucket/folders here if you wish.
bucket = sess.default_bucket()
folder = "DEMO-Sagemaker-Edge"
compilation_output_sub_folder = folder + "/compilation-output"
iot_folder = folder + "/iot"

# S3 Location to save the model artifact after compilation
s3_compilation_output_location = "s3://{}/{}".format(bucket, compilation_output_sub_folder)

print('Role: ', role.split('/')[-1])

<span style="color:red">**Locate the above printed sagemaker role from [IAM console](https://console.aws.amazon.com/iam), find and attach the following policies to role:**</span>

- IAMFullAccess

<span style="color:red">**Also add the following trust relationship**</span>
```
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "sagemaker.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    },
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "iam.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
```

In [ ]:
## setup all the IAM and Policies needed
from setup_utils import setup_roles_and_policies

iot_device_role_name = 'SageMakerTESRole' + str(time.time()).split(".")[0]
ec2_role_name = setup_roles_and_policies(iot_device_role_name)
print('Done')

### Launch EC2 Instance

As mentioned earlier, this EC2 instance is used in place of an Edge device for running the agent software. Generate key pair for EC2 instance, save the key PEM file. We can use this key with SSH to connect to the instance. But in this notebook example, we will not use SSH, instead, we will use AWS Systems Manager to send commands to the instance.

In [ ]:
ec2_client = boto3.client("ec2", region_name=region)

key_pairs = ec2_client.describe_key_pairs()
key_names = list(map(lambda x: x["KeyName"], key_pairs["KeyPairs"]))

key_name = "ec2-key-pair"

if key_name in key_names:
    ec2_key_pair = ec2_client.delete_key_pair(
        KeyName=key_name,
    )

In [ ]:
ec2_key_pair = ec2_client.create_key_pair(
    KeyName=key_name,
)

try:
    key_pair = str(ec2_key_pair["KeyMaterial"])
    key_pair_file = open("ec2-key-pair.pem", "w")
    key_pair_file.write(key_pair)
    key_pair_file.close()
except:
    pass

print('Done')

Launch an EC2 C5 instance. In this example we will use aws deep learning AMI.

In [ ]:
ami = ec2_client.describe_images(Filters=[{'Name': 'name', 'Values': ['Deep Learning AMI (Ubuntu 18.04) Version 36.0']}])['Images'][0]['ImageId']

ec2_instance = ec2_client.run_instances(
    ImageId=ami,
    MinCount=1,
    MaxCount=1,
    InstanceType="c5.large",
    KeyName=key_name,
    IamInstanceProfile={"Name": ec2_role_name},
)

instance_id = ec2_instance["Instances"][0]["InstanceId"]  # will used for running inference later
print(instance_id)

## Compiling/Packaging/Deploying our ML model to our edge devices

In this below section you will :

 - Compile/Optimize your pre-trained model to your edge device (Linux X86_64) using [SageMaker NEO](https://docs.aws.amazon.com/sagemaker/latest/dg/neo.html)
 - Create a deployment package with a signed model + the runtime used by SageMaker Edge Agent to load and invoke the optimized model
 - Deploy the package using IoT Jobs


Invoking SageMaker NEO to compile the pre-trained model. To know how this model was trained please refer to the training notebook [here](https://github.com/aws-samples/amazon-sagemaker-edge-manager-workshop/blob/main/lab/02%20-%20Training%20with%20Pytorch.ipynb). 

Upload the pre-trained model to S3 bucket

In [ ]:
model_file = open("model/model.tar.gz", "rb")
boto3.Session().resource("s3").Bucket(bucket).Object('model/model.tar.gz').upload_fileobj(model_file)
print("Model successfully uploaded!")

It will compile the model for targeted hardware and OS with SageMaker Neo service. It will also include the [deep learning runtime](https://github.com/neo-ai/neo-ai-dlr) in the model package.

In [ ]:
sm_client = boto3.client("sagemaker", region_name=region)
compilation_job_name = 'wind-turbine-anomaly-%d' % int(time.time()*1000)
sm_client.create_compilation_job(
    CompilationJobName=compilation_job_name,
    RoleArn=role,
    InputConfig={
        'S3Uri': 's3://%s/model/model.tar.gz' % bucket,
        'DataInputConfig': '{"input0":[1,6,10,10]}',
        'Framework': 'PYTORCH'
    },
    OutputConfig={
        'S3OutputLocation': 's3://%s/wind_turbine/optimized/' % bucket,        
        'TargetPlatform': { 'Os': 'LINUX', 'Arch': 'X86_64' }
    },
    StoppingCondition={ 'MaxRuntimeInSeconds': 900 }
)
while True:
    resp = sm_client.describe_compilation_job(CompilationJobName=compilation_job_name)    
    if resp['CompilationJobStatus'] in ['STARTING', 'INPROGRESS']:
        print('Running...')
    else:
        print(resp['CompilationJobStatus'], compilation_job_name)
        break
    time.sleep(5)

### Package Keras Model

It will sign the model and create a deployment package with:
1. The optimized model
1. Model Metadata
1. Create a Greengrass V2 model component


In [ ]:
import time
model_version = '1.0'
model_name = 'WindTurbineAnomalyDetection'
model_component_version = "1.0.0"
model_component_name = "com.model.WindTurbineAnomalyDetection"

# remove component if it is already exists
ggv2_client = boto3.client('greengrassv2')
account_id = role.split(":")[4]
try:
    ggv2_client.delete_component(arn='arn:aws:greengrass:{}:{}:components:{}:versions:{}'.format(region, account_id, model_component_name, model_component_version))
except:
    print('component does not exists')
    

# edge package job
edge_packaging_job_name='wind-turbine-anomaly-%d' % int(time.time()*1000)
resp = sm_client.create_edge_packaging_job(
    EdgePackagingJobName=edge_packaging_job_name,
    CompilationJobName=compilation_job_name,
    ModelName=model_name,
    ModelVersion=model_version,
    RoleArn=role,
    OutputConfig={
        "PresetDeploymentType": "GreengrassV2Component",
        "PresetDeploymentConfig": json.dumps(
            {"ComponentName": model_component_name, "ComponentVersion": model_component_version}
        ),
        'S3OutputLocation': s3_compilation_output_location
    }
)

# Poll every 30 sec
while True:
    job_status = sm_client.describe_edge_packaging_job(
        EdgePackagingJobName=edge_packaging_job_name
    )
    if job_status["EdgePackagingJobStatus"] == "COMPLETED":
        break
    elif job_status["EdgePackagingJobStatus"] == "FAILED":
        raise RuntimeError("Edge Packaging failed")
    print("Packaging ...")
    time.sleep(30)
    
print('Done')


### Setup Greengrass

SageMaker Edge Manager can use AWS IoT Greengrass to deploy the agent, the model and the inference application to the edge device.

AWS IoT Greengrass provides all the necessary features to manage applications on remote devices in a secure and scalable way. To learn more about Greengrass, head to the [What is AWS IoT Greengrass?](https://docs.aws.amazon.com/greengrass/v2/developerguide/what-is-iot-greengrass.html). 

In [ ]:
ssm_client = boto3.client("ssm", region_name=region)
device_name = "demo-device" + str(time.time()).split(".")[0]

In [ ]:
response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "#!/bin/bash",
            "sudo apt update",
            "sudo apt-get install python3-venv -y",
            "curl -s https://d2s8p88vqu9w66.cloudfront.net/releases/greengrass-nucleus-latest.zip > greengrass-nucleus-latest.zip && unzip greengrass-nucleus-latest.zip -d GreengrassCore",
            f'sudo -E java -Droot="/greengrass/v2" -Dlog.store=FILE -jar ./GreengrassCore/lib/Greengrass.jar --aws-region {region} --thing-name {device_name} --tes-role-name {iot_device_role_name} --tes-role-alias-name {iot_device_role_name}Alias --thing-group-name GreengrassSMEdgeManagerGroup --component-default-user ggc_user:ggc_group --provision true --setup-system-service true --deploy-dev-tools true',
        ]
    },
)

Need to wait for one or two minutes before below command will show `'Status': 'Success'`

In [ ]:
time.sleep(100)
ssm_client.get_command_invocation(
    CommandId=response["Command"]["CommandId"],
    InstanceId=instance_id,
)

#### Create Edge Manager Device Fleet

In [ ]:
from setup_utils import modify_device_role

role_arn = modify_device_role(iot_device_role_name)

device_fleet_name = "demo-device-fleet" + str(time.time()).split(".")[0]

sm_client.create_device_fleet(
    DeviceFleetName=device_fleet_name,
    RoleArn=role_arn,
    OutputConfig={"S3OutputLocation": "s3://{}".format(bucket)}
)

print(device_fleet_name)

#### Register device to the fleet

In [ ]:
sm_client.register_devices(
    DeviceFleetName=device_fleet_name,
    Devices=[
        {
            "DeviceName": device_name,
            "IotThingName": device_name,
            "Description": "this is a sample virtual device",
        }
    ],
)

print('Done')

### Setup Wind turbine application as Greengrass component

Next, upload the python script, dataset and the protobuf libraries to an S3 bucket so that they can be referenced by the component recipe and from where they will be downloaded by Greengrass on the device. We use the SageMaker bucket for this.

In [ ]:
!mkdir -p data
!curl https://aws-ml-blog.s3.amazonaws.com/artifacts/monitor-manage-anomaly-detection-model-wind-turbine-fleet-sagemaker-neo/dataset_wind_turbine.csv.gz -o data/dataset_wind.csv.gz

In [ ]:
folder = "com.sagemaker.windturbine/1.0.0"
sess.upload_data("app_ggv2/entry_point.py", bucket, folder)
sess.upload_data("app_ggv2/agent_pb2.py", bucket, folder)
sess.upload_data("app_ggv2/agent_pb2_grpc.py", bucket, folder)
sess.upload_data("app_ggv2/edgeagentclient.py", bucket, folder)
sess.upload_data("app_ggv2/turbine.py", bucket, folder)
sess.upload_data("app_ggv2/windfarm.py", bucket, folder)
sess.upload_data("data/dataset_wind.csv.gz", bucket, folder)
sess.upload_data("statistics/raw_std.npy", bucket, folder)
sess.upload_data("statistics/mean.npy", bucket, folder)
sess.upload_data("statistics/std.npy", bucket, folder)
sess.upload_data("statistics/thresholds.npy", bucket, folder)

recipe = f'''
---
RecipeFormatVersion: 2020-01-25
ComponentName: com.sagemaker.windturbine
ComponentVersion: 1.0.0
ComponentDescription: Sagemaker Edge Manager Python example
ComponentPublisher: Amazon Web Services, Inc.
ComponentDependencies:
  aws.greengrass.SageMakerEdgeManager:
    VersionRequirement: '>=1.0.0'
    DependencyType: HARD
  com.model.WindTurbineAnomalyDetection:
    VersionRequirement: '~1.0.0'
    DependencyType: HARD
Manifests:
  - Platform:
      os: linux
      architecture: "/amd64|x86/"
    Lifecycle:
      Install: |-
        python3 -m venv venv
        . venv/bin/activate
        pip install pip --upgrade
        pip install wheel      
        pip3 install grpcio==1.38.1
        pip3 install grpcio-tools==1.38.1
        pip3 install protobuf
        pip3 install Pillow
        pip3 install numpy
        pip3 install pandas
        pip3 install PyWavelets
      Run:
        Script: |- 
          export MODEL_PATH="{{com.model.WindTurbineAnomalyDetection:work:path}}"
          export ARTIFACTS_PATH="{{artifacts:path}}"
          . venv/bin/activate  
          python3 -u {{artifacts:path}}/entry_point.py 
    Artifacts:
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/entry_point.py
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/agent_pb2.py
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/agent_pb2_grpc.py
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/edgeagentclient.py
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/turbine.py
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/windfarm.py
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/dataset_wind.csv.gz
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/raw_std.npy
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/mean.npy
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/std.npy
      - URI: s3://{bucket}/com.sagemaker.windturbine/1.0.0/thresholds.npy
'''

print(recipe)

#### As a final step, create the component. You can use the [AWS Greengrass Console](https://console.aws.amazon.com/iot/home?#/greengrass/v2/components/create) to create a new component. Go to **Component>Create component**, select `Enter recipe as YAML` and copy and paste the YAML above. 

# Deploy the application

In [ ]:
## Run this cell in case some of the dependencies was not setup correctly on the EC2
cat_log_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "sudo apt-get install python3-venv -y",
        ]
    },
)

time.sleep(15)
output = ssm_client.get_command_invocation(
    CommandId=cat_log_out["Command"]["CommandId"],
    InstanceId=instance_id,
)["StandardOutputContent"]
print(output)

Once the application component has been created, it can be deployed to the device. 

1. In the [AWS Console](https://console.aws.amazon.com/iot/home#/greengrass/v2/components/private) select the `com.sagemaker.windturbine` component, and choose **Deploy**.
2. Select the deployment named `Deployment for GreengrassSMEdgeManagerGroup` and choose `Next`
3. Toggle the selector on the `Public components` pane, search for `SageMakerEdgeManager`, and selct it
4. Choose **Next**
5. Select the `aws.greengrass.SageMakerEdgeManager` component and choose **Configure component**
6. Replace the **Configuration to merge** content with the following json. Don't forget to change the placeholder to the actual values.
```json
{
	"DeviceFleetName": <device-fleet-name>,
	"BucketName": <bucket-name>
}
```
7. Choose **Confirm**
3. Choose **Next** until you reach the last screen. 
4. Choose **Deploy**.

You can check the status of the deployment by clicking on the device name `GreengrassSMEdgeManagerDevice` in the Target core devices pane and then selecting Deployments tab.

In [ ]:
cat_log_out = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "sudo tail -30 /greengrass/v2/logs/com.sagemaker.windturbine.log",
        ]
    },
)

In [ ]:
output = ssm_client.get_command_invocation(
    CommandId=cat_log_out["Command"]["CommandId"],
    InstanceId=instance_id,
)["StandardOutputContent"]
print(output)

## Clean Up

Stop the EC2 instance

In [ ]:
ec2_client.stop_instances(InstanceIds=[instance_id])

Deregister device and delete device fleet

In [ ]:
sm_client.deregister_devices(DeviceFleetName=device_fleet_name, DeviceNames=[device_name])

sm_client.delete_device_fleet(DeviceFleetName=device_fleet_name)